In [1]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder'    
except ModuleNotFoundError:
    pass

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder


In [2]:
# Imports
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed

import tensorflow as tf

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import binary_crossentropy
from keras import backend as K

try:
    import face_recognition
except ModuleNotFoundError:
    !pip install face_recognition

from Util import pipeline
from Baseline.classifiers import *

In [ ]:
# General model settings
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 256, 256, 3
EPOCHS = 100
DATA_GENERATOR_SEED = 1337
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2

tf.random.set_seed(DATA_GENERATOR_SEED)
seed(DATA_GENERATOR_SEED)

# Pick dataset; DF_TYPE={'rnd', 'avg'}
for DF_TYPE in ['diff', 'avg', 'rnd'][::-1]:
    TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30-test'

    TRAIN_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, horizontal_flip = True, fill_mode='nearest', validation_split = VALIDATION_SPLIT)
    TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                        batch_size = BATCH_SIZE,
                                                        class_mode = 'binary', 
                                                        target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                        subset = 'training',
                                                        seed = DATA_GENERATOR_SEED,
                                                        follow_links = True)

    VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, validation_split = VALIDATION_SPLIT)
    VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                             batch_size = BATCH_SIZE,
                                                             class_mode = 'binary', 
                                                             target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                             subset = 'validation',
                                                             seed = DATA_GENERATOR_SEED,
                                                             follow_links = True)

    # Train MesoInception4 w/ F2F weights
    TEST_MODEL = MesoInception4().model
    TEST_MODEL.compile(optimizer = TEST_MODEL.optimizer,
                       loss = TEST_MODEL.loss,
                       metrics = TEST_MODEL.metrics + [metrics.BinaryAccuracy(name = 'acc'),
                                                       metrics.AUC(name = 'auc'),
                                                       metrics.FalsePositives(name = 'fp')])
    
    WEIGHTS_PATH = './Baseline/weights/MesoInception_F2F.h5'
    TEST_MODEL.load_weights(WEIGHTS_PATH) 

    # Make last layer trainable
    for layer in TEST_MODEL.layers:
        layer.trainable = False
    TEST_MODEL.layers[-1].trainable = True

    train_gen_list = list(TRAIN_GENERATOR.classes)
    val_gen_list = list(VALIDATION_GENERATOR.classes)

    train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
    val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)

    pos = train_pos + val_pos
    neg = train_neg + val_neg
    total = pos + neg

    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0

    CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

    EARLY_STOP = EarlyStopping(monitor='val_auc',
                            patience=EPOCHS//20,
                            mode='max',
                            verbose=1,
                            restore_best_weights=True)

    HISTORY = TEST_MODEL.fit(TRAIN_GENERATOR,
                            steps_per_epoch = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size,
                            validation_data = VALIDATION_GENERATOR,
                            validation_steps = VALIDATION_GENERATOR.n//VALIDATION_GENERATOR.batch_size,
                            epochs = EPOCHS,
                            verbose = 1,
                            class_weight = CLASS_WEIGHT,
                            callbacks = [EARLY_STOP])

    weight_name = WEIGHTS_PATH.split('_')[-1].replace('.h5', '')
    pipeline.evaluate_model(TEST_MODEL = TEST_MODEL,
                            EXPERIMENT_NAME = f'Minimally Trained MesoInception4 on {DF_TYPE} with {weight_name}',
                            TEST_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30-test',
                            WEIGHTS_PATH = '',
                            HISTORY = HISTORY)

Found 15711 images belonging to 2 classes.
Found 3927 images belonging to 2 classes.
Epoch 1/100
107/122 [=========================>....] - ETA: 19:41 - loss: 0.3153 - acc: 0.5490 - auc: 0.5129 - fp: 258.1682